<a href="https://colab.research.google.com/github/hashk1/nlp-100-knock-2020-rev2/blob/main/03-%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%8F%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第3章: 正規表現

In [ ]:
# ライブラリ読み込み
import pandas as pd
import re
import requests
import json
from IPython.display import SVG

In [ ]:
# データ取得
! wget -c https://nlp100.github.io/data/jawiki-country.json.gz

### 20. JSONデータの読み込み

In [ ]:
pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0]

### 21. カテゴリ名を含む行を抽出

In [ ]:
[x for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n") if "[[Category:" in x]

### 22. カテゴリ名の抽出

In [ ]:
[x.split(":")[1].split("|")[0].replace("]", "") for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n") if "[[Category:" in x]

### 23. セクション構造

In [ ]:
[(x.replace("=", ""), x.count("=") // 2 - 1) for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n") if "==" in x]

### 24. ファイル参照の抽出

In [ ]:
[x.split(":")[1].split("|")[0].replace("]", "").replace("}", "") for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n") if "[ファイル:" in x]

### 25. テンプレートの抽出

In [ ]:
d = {}
for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n"):
    if "==" in x: break
    if not x.startswith("|"): continue
    y = x[1:].split("=")
    if len(y) < 2: continue
    d[y[0].strip()] = "=".join(y[1:]).strip()
d

### 26. 強調マークアップの除去

In [ ]:
d = {}
for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n"):
    if "==" in x: break
    if not x.startswith("|"): continue
    y = x[1:].split("=")
    if len(y) < 2: continue
    d[y[0].strip()] = "=".join(y[1:]).strip().replace("'", "")
d

### 27. 内部リンクの除去

In [ ]:
d = {}
for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n"):
    if "==" in x: break
    if not x.startswith("|"): continue
    y = x[1:].split("=")
    if len(y) < 2: continue
    yy = "=".join(y[1:]).strip().replace("'", "")
    yy = re.subn("\[\[[^\]\{]*\|", "",  yy, 1000)[0]
    yy = yy.replace("[[", "").replace("]]", "")
    d[y[0].strip()] = yy
d

### 28. MediaWikiマークアップの除去

In [ ]:
d = {}
for x in pd.read_json("jawiki-country.json.gz", lines=True).query("title == 'イギリス'")["text"].values[0].split("\n"):
    if "==" in x: break
    if not x.startswith("|"): continue
    y = x[1:].split("=")
    if len(y) < 2: continue
    yy = "=".join(y[1:]).strip().replace("'", "")
    yy = re.subn("\[\[ファイル:[^\]]*\]\]", "",  yy, 1000)[0]
    yy = re.subn("\[\[[^\]\{]*\|", "",  yy, 1000)[0]
    yy = yy.replace("[[", "").replace("]]", "")
    yy = re.subn("<ref(.)*\/ref>", "",  yy, 1000)[0]
    yy = re.subn("<ref(.)*\/>", "",  yy, 1000)[0]
    yy = yy.replace("{{0}}", "")
    yy = re.subn("\{\{[^\}\[]*\|", "",  yy, 1000)[0]
    yy = yy.replace("{{", "").replace("}}", "")
    yy = yy.replace("<br", "").replace("/>", "")
    d[y[0].strip()] = yy
d

### 29. 国旗画像のURLを取得する

In [ ]:
url = "https://commons.wikimedia.org/w/api.php?action=query&titles=File:{}&prop=imageinfo&iiprop=url&format=json".format(d["国旗画像"])
img_json = json.loads(requests.get(url).text)
key = [k for k in img_json["query"]["pages"].keys()][0]
img_url = img_json["query"]["pages"][key]["imageinfo"][0]["url"]
img_url

In [ ]:
SVG(img_url)